In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.8.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: msrest, azure-mgmt-core, tqdm, azure-storage-blob, pydash, opencensus-ext-azure, marshmallow, azure-core, colorama, azure-storage-file-share, azure-storage-file-datalake, pyjwt, azure-common, typing-extensions, pyyaml, isodate, strictyaml, jsonschema
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [4]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [ ]:
!az ml data create --name diabetes-dev-folder --version 1 --path "experimentation/data/" --type uri_folder

In [14]:
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes
# configure input and output
my_job_inputs = {
    "local_data": Input(type=AssetTypes.URI_FOLDER, path="azureml:diabetes-dev-folder:1")
}

In [15]:
from azure.ai.ml import command

# configure job
job = command(
    code="./model",
    command="python train.py --training_data ${{inputs.local_data}} --reg_rate 0.01",
    inputs=my_job_inputs,
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-mslearn-mlops-display",
    experiment_name="diabetes-mslearn-mlops"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/plucky_nut_6ysy0jjhgl?wsid=/subscriptions/74a47057-1fd0-47a1-b841-743f703fb07d/resourcegroups/rg-dp100-l3fecad3dc65b4b94b6/workspaces/mlw-dp100-l3fecad3dc65b4b94b6&tid=c7b4f3a8-b75e-4b78-b8b9-b56023be76aa


In [ ]:
!az ml job create --file job.yml